# Создание агентов для игры "Камень, ножницы, бумага"
Требуется создасть агентов для игры с разными стратегиями и провести с ними турнир на выявление лучшей стратегии.
Правила турнира:
* все играют один раз против всех одну партию
* партия включает 10 игр по 10 ходов
* итоги каждой игры суммируются по кол-ву баллов. Если по итогам игры один из игроков набирает большее кол-во баллов, то он выигрывает игру (won=1) и наоборот
* победителем партии считается игрок, набравший большее кол-во побед по итогам 10 игр партии
* победителем игры считается игрок, победивший в большем кол-ве партий


In [1]:
!pip install -q -U kaggle_environments

In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Создадим агентов в виде функций. Краткое описание стратегии агентов в комментариях к каждому из них. 

In [3]:
%%writefile rock_agent.py
#выкидывает только камень
def rock_agent(observation, configuration):
    return 0     

Overwriting rock_agent.py


In [4]:
%%writefile paper_agent.py
#выкидывает только бумагу
def paper_agent(observation, configuration):
    return 1 

Overwriting paper_agent.py


In [5]:
%%writefile scissors_agent.py
#выкидывает только ножнцы
def scissors_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [6]:
%%writefile random_agent.py
#случайный выбор 
def random_agent(observation, configuration):
    import random
    return random.randint(0,2)

Overwriting random_agent.py


In [7]:
%%writefile random_agent2.py
# случайный выбор - другой подход к генерации через список и шафлл списка
def random_agent2(observation, configuration):
    import random
    a = [i for i in range(0,8)]
    random.shuffle(a)
    x=int(a.pop())
    return x%3

Overwriting random_agent2.py


In [8]:
%%writefile copycat_agent.py
# повторяет предыдущий ход оппонента 
def copycat_agent(observation, configuration):
    if observation.step>0:
        return observation.lastOpponentAction
    else:
        import random
        return random.randint(0,2)

Overwriting copycat_agent.py


In [9]:
%%writefile opposite_agent.py
# выкидывает противоположное значение предыдущему ходу оппонента
def opposite_agent(observation, configuration):
    if observation.step>0:
        if observation.lastOpponentAction == 0:
            return 1
        elif observation.lastOpponentAction == 1:
            return 2
        else:
            return 0
        
    else:
        import random
        return random.randint(0,2)

Overwriting opposite_agent.py


In [10]:
%%writefile fixed_pattern_agent.py
# ходит по фиксированному шаблону: камень на кратные 3-м хода, ножницы на кратные 2, на остальные - бумага
def fixed_pattern_agent(observation, configuration):
    if observation.step == 0:
        return 0
    elif observation.step%3 == 0:
        return 0
    elif observation.step%2 == 0:
        return 2
    else:
        return 1

Overwriting fixed_pattern_agent.py


In [11]:
%%writefile rock_paper_agent.py
#чередует камень и бумагу
def rock_paper_agent(observation, configuration):
    if observation.step == 0:
        return 1
    elif observation.step%2 == 0:
        return 1
    else:
        return 0

Overwriting rock_paper_agent.py


In [12]:
%%writefile rock_scissors_agent.py
#чередует камень и ножницы
def rock_scissors_agent(observation, configuration):
    if observation.step == 0:
        return 2
    elif observation.step%2 == 0:
        return 2
    else:
        return 0

Overwriting rock_scissors_agent.py


In [13]:
%%writefile paper_scissors_agent.py
#чередует ножницы и бумагу
def paper_scissors_agent(observation, configuration):
    if observation.step == 0:
        return 2
    elif observation.step%2 == 0:
        return 2
    else:
        return 1

Overwriting paper_scissors_agent.py


In [14]:
%%writefile copycat_fixed_pattern_agent.py
#корирует послднее действие оппонента со сдвигом по фиксированному шаблону: кратные 2 ходы сдвиг на 1, кратные 3 - сдвиг на 2
def copycat_fixed_pattern_agent(observation, configuration):
    if observation.step == 0:
        import random
        return random.randint(0,2)
    elif observation.step%3 == 0:
        return (observation.lastOpponentAction + 2) % 3
    elif observation.step%2 == 0:
        return (observation.lastOpponentAction + 1) % 3
    else:
        return observation.lastOpponentAction

Overwriting copycat_fixed_pattern_agent.py


In [15]:
%%writefile copycat_shift2.py
#копирует последнее действие оппонета со сдвигом на 2
def copycat_shift2(observation, configuration):
    if observation.step>0:
        return (observation.lastOpponentAction + 2) % 3
        
    else:
        import random
        return random.randint(0,2)

Overwriting copycat_shift2.py


In [16]:
%%writefile copycat_shift1.py
#копирует последнее действие оппонета со сдвигом на 1
def copycat_shift1(observation, configuration):
    if observation.step>0:
        return (observation.lastOpponentAction + 1) % 3
        
    else:
        import random
        return random.randint(0,2)

Overwriting copycat_shift1.py


In [17]:
#создадим список агентов для участия в турнире
list_names = [
    "rock_agent", 
    "paper_agent",
    "random_agent",
    "copycat_agent",
    "copycat_shift1",
    "copycat_shift2",
    "copycat_fixed_pattern_agent",
    "paper_scissors_agent",
    "rock_scissors_agent",
    "rock_paper_agent",
    "fixed_pattern_agent",
    "opposite_agent",
    "random_agent2",
    "scissors_agent",   

]
list_agents = [agent_name + ".py" for agent_name in list_names]

In [18]:
# турнир по принципу играют все против всех по 10 партий в 10 шагов. За выигрыш в партии - 1 очко
env = make("rps", configuration = {'episodeSteps':10})
print("Simulation of battles. Wait...")
total_score={}
battle_results={}
for ind_agent_1 in range(0,len(list_names)):
    for ind_agent_2 in range(ind_agent_1+1, len(list_names)):
        print(f"Log: {list_names[ind_agent_1]} vs {list_names[ind_agent_2]}")
        
      
        current_score = evaluate(
            "rps", 
            [list_agents[ind_agent_1] , list_agents[ind_agent_2]], 
            num_episodes=10
        )
        agents=str(list_names[ind_agent_1] +' '+'vs'+' '+ list_names[ind_agent_2])
        won = 0
        lost = 0
        tie = 0
        for i in current_score:
            score = i[0]
            if score > 0:
                won += 1 
            elif score < 0:
                lost += 1
            else: 
                tie += 1 
        battle_results[agents]={}
        battle_results[agents]['won'] = won
        battle_results[agents]['lost'] = lost
        battle_results[agents]['tie'] = tie
   
        
        if won > lost:
            if list_agents[ind_agent_1] in total_score:
                total_score[list_agents[ind_agent_1]]+=1
            if list_agents[ind_agent_1] not in total_score:
                total_score[list_agents[ind_agent_1]]=total_score.get(list_agents[ind_agent_1],0)+1
        if won < lost:
            if list_agents[ind_agent_2] in total_score:
                total_score[list_agents[ind_agent_2]]+=1
            if list_agents[ind_agent_2] not in total_score:
                total_score[list_agents[ind_agent_2]]=total_score.get(list_agents[ind_agent_2],0)+1       

Simulation of battles. Wait...
Log: rock_agent vs paper_agent
Log: rock_agent vs random_agent
Log: rock_agent vs copycat_agent
Log: rock_agent vs copycat_shift1
Log: rock_agent vs copycat_shift2
Log: rock_agent vs copycat_fixed_pattern_agent
Log: rock_agent vs paper_scissors_agent
Log: rock_agent vs rock_scissors_agent
Log: rock_agent vs rock_paper_agent
Log: rock_agent vs fixed_pattern_agent
Log: rock_agent vs opposite_agent
Log: rock_agent vs random_agent2
Log: rock_agent vs scissors_agent
Log: paper_agent vs random_agent
Log: paper_agent vs copycat_agent
Log: paper_agent vs copycat_shift1
Log: paper_agent vs copycat_shift2
Log: paper_agent vs copycat_fixed_pattern_agent
Log: paper_agent vs paper_scissors_agent
Log: paper_agent vs rock_scissors_agent
Log: paper_agent vs rock_paper_agent
Log: paper_agent vs fixed_pattern_agent
Log: paper_agent vs opposite_agent
Log: paper_agent vs random_agent2
Log: paper_agent vs scissors_agent
Log: random_agent vs copycat_agent
Log: random_agent vs 

In [19]:
sorted_total_score = sorted(total_score.items(), key=lambda item: item[1], reverse=True)

print('Подсчет итогов', sorted_total_score)

Подсчет итогов [('random_agent2.py', 8), ('paper_scissors_agent.py', 7), ('copycat_shift1.py', 6), ('rock_scissors_agent.py', 6), ('copycat_shift2.py', 6), ('paper_agent.py', 5), ('opposite_agent.py', 5), ('fixed_pattern_agent.py', 5), ('random_agent.py', 4), ('rock_agent.py', 3), ('rock_paper_agent.py', 3), ('scissors_agent.py', 3), ('copycat_fixed_pattern_agent.py', 3), ('copycat_agent.py', 1)]


Агент(ы), набравшие большее кол-во баллов, являются лучшими по итогам турнира

In [20]:
#справочно - вывод результатов партий
print('Результаты турнира',battle_results)

Результаты турнира {'rock_agent vs paper_agent': {'won': 0, 'lost': 10, 'tie': 0}, 'rock_agent vs random_agent': {'won': 3, 'lost': 3, 'tie': 4}, 'rock_agent vs copycat_agent': {'won': 0, 'lost': 0, 'tie': 10}, 'rock_agent vs copycat_shift1': {'won': 0, 'lost': 10, 'tie': 0}, 'rock_agent vs copycat_shift2': {'won': 10, 'lost': 0, 'tie': 0}, 'rock_agent vs copycat_fixed_pattern_agent': {'won': 0, 'lost': 0, 'tie': 10}, 'rock_agent vs paper_scissors_agent': {'won': 0, 'lost': 0, 'tie': 10}, 'rock_agent vs rock_scissors_agent': {'won': 10, 'lost': 0, 'tie': 0}, 'rock_agent vs rock_paper_agent': {'won': 0, 'lost': 10, 'tie': 0}, 'rock_agent vs fixed_pattern_agent': {'won': 0, 'lost': 0, 'tie': 10}, 'rock_agent vs opposite_agent': {'won': 0, 'lost': 10, 'tie': 0}, 'rock_agent vs random_agent2': {'won': 0, 'lost': 10, 'tie': 0}, 'rock_agent vs scissors_agent': {'won': 10, 'lost': 0, 'tie': 0}, 'paper_agent vs random_agent': {'won': 3, 'lost': 2, 'tie': 5}, 'paper_agent vs copycat_agent': {'w